# 记录 deepspeed
源码编译由于网络环境安装总失败
采用 deepspeed 的github上建议使用pip 安装
安装环境
```shell
## 建立隔离环境
cd DeepSpedd
conda env create -n tf -f environment.yml
conda activate df
pip install deepspeed
## 安装缺失包
apt install htop
pip install datasets
pip install evaluate
pip install transformers
pip install transformers[torch]
pip install sacrebleu

## 训练还需要几个额外包
cd ../LLM-quickstart/deepspeed/translation
pip install -r requirements.txt
```

## 教程中的脚本运行时报错
```
[rank0]: Exception: Current loss scale already at minimum - cannot decrease scale anymore. Exiting run.
```
### 直接问qwen3, 贴入足够多的报错日志获得解决

- 你遇到的 DeepSpeed 训练报错是典型的FP16 溢出（overflow）问题，最终导致训练中断。以下是关键信息的解析和解决方案：
- 使用 BF16 替代 FP16（推荐）
- 对应命令行去掉 --fp16，加上 --bf16。

训练脚本
```shell
# DeepSpeed ZeRO-3
deepspeed --num_gpus=1 translation/run_translation.py \
--deepspeed config/ds_qwen.json \
--model_name_or_path t5-3b --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --bf16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro

```

### 截图
#### 训练中
![加载QLoRA](./2025-08-07_103244.png)

#### htop 截图
![加载QLoRA](./2025-08-07_103258.png)

#### GPU截图
![加载QLoRA](./2025-08-07_103817.png)


## 训练完成
```shell
[INFO|trainer.py:2433] 2025-08-07 02:01:31,531 >> ***** Running training *****
[INFO|trainer.py:2434] 2025-08-07 02:01:31,532 >>   Num examples = 500
[INFO|trainer.py:2435] 2025-08-07 02:01:31,532 >>   Num Epochs = 1
[INFO|trainer.py:2436] 2025-08-07 02:01:31,532 >>   Instantaneous batch size per device = 1
[INFO|trainer.py:2439] 2025-08-07 02:01:31,532 >>   Total train batch size (w. parallel, distributed & accumulation) = 1
[INFO|trainer.py:2440] 2025-08-07 02:01:31,532 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2441] 2025-08-07 02:01:31,532 >>   Total optimization steps = 500
[INFO|trainer.py:2442] 2025-08-07 02:01:31,540 >>   Number of trainable parameters = 2,851,569,664
{'loss': 0.4943, 'grad_norm': 10.869831085205078, 'learning_rate': 5e-05, 'epoch': 1.0}                                                                                    
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [39:34<00:00,  4.54s/it][INFO|trainer.py:4074] 2025-08-07 02:41:09,769 >> Saving model checkpoint to output_dir/checkpoint-500
[INFO|configuration_utils.py:478] 2025-08-07 02:41:09,775 >> Configuration saved in output_dir/checkpoint-500/config.json
[INFO|configuration_utils.py:869] 2025-08-07 02:41:09,776 >> Configuration saved in output_dir/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:4180] 2025-08-07 02:41:16,675 >> The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at output_dir/checkpoint-500/model.safetensors.index.json.
[INFO|tokenization_utils_base.py:2562] 2025-08-07 02:41:16,677 >> tokenizer config file saved in output_dir/checkpoint-500/tokenizer_config.json
[INFO|tokenization_utils_base.py:2571] 2025-08-07 02:41:16,677 >> Special tokens file saved in output_dir/checkpoint-500/special_tokens_map.json
[2025-08-07 02:41:16,912] [INFO] [logging.py:107:log_dist] [Rank 0] [Torch] Checkpoint global_step500 is begin to save!
[2025-08-07 02:41:16,934] [INFO] [logging.py:107:log_dist] [Rank 0] Saving model checkpoint: output_dir/checkpoint-500/global_step500/zero_pp_rank_0_mp_rank_00_model_states.pt
[INFO|trainer.py:2718] 2025-08-07 02:41:57,924 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 2426.384, 'train_samples_per_second': 0.206, 'train_steps_per_second': 0.206, 'train_loss': 0.4943197021484375, 'epoch': 1.0}                            
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [40:26<00:00,  4.85s/it]
[INFO|trainer.py:4074] 2025-08-07 02:42:01,432 >> Saving model checkpoint to output_dir
[INFO|configuration_utils.py:478] 2025-08-07 02:42:01,439 >> Configuration saved in output_dir/config.json
[INFO|configuration_utils.py:869] 2025-08-07 02:42:01,439 >> Configuration saved in output_dir/generation_config.json
[INFO|modeling_utils.py:4180] 2025-08-07 02:42:09,760 >> The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at output_dir/model.safetensors.index.json.
[INFO|tokenization_utils_base.py:2562] 2025-08-07 02:42:09,766 >> tokenizer config file saved in output_dir/tokenizer_config.json
[INFO|tokenization_utils_base.py:2571] 2025-08-07 02:42:09,767 >> Special tokens file saved in output_dir/special_tokens_map.json
***** train metrics *****
  epoch                    =        1.0
  total_flos               =        3GF
  train_loss               =     0.4943
  train_runtime            = 0:40:26.38
  train_samples            =        500
  train_samples_per_second =      0.206
  train_steps_per_second   =      0.206
[INFO|modelcard.py:456] 2025-08-07 02:42:11,288 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Translation', 'type': 'translation'}, 'dataset': {'name': 'wmt16 ro-en', 'type': 'wmt16', 'args': 'ro-en'}}
[2025-08-07 02:42:26,532] [INFO] [launch.py:351:main] Process 8586 exits successfully.

```